# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [3]:
document = ET.parse( './data/mondial_database.xml' )

In [4]:
import pandas as pd

1) 10 countries with the lowest infant mortality rates

In [8]:
root = document.getroot()
country_ = []
infant_mortality_ = []
for country in root.iter('country'):
    if country.find('name')!= None :
        if country.find('infant_mortality') != None:
            country_.append(country.find('name').text)
            infant_mortality_.append(float(country.find('infant_mortality').text))
        else:
            continue
    else:
        continue

df = {'country':country_, 'infant mortality': infant_mortality_}
low_infant_mortality_rates = pd.DataFrame(df).sort_values(by='infant mortality').head(10)
low_infant_mortality_rates

,country,infant mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


2) 10 cities with the largest population

In [11]:
cities = []
city_populations = []
population_=[]

for country in document.findall('country'):
    for node in country.getiterator():
        for city in node.findall('city'):
           
            for population in city.findall('population'):
                population_.append(int(population.text))
            if(len(population_) > 0):
                city_populations.append(population_[len(population_)-1])
                cities.append(city.find('name').text)
            
pop_df = pd.DataFrame({'city_name' : cities,'population' : city_populations })
pop_df['population'] = pop_df['population'].astype(int)
pop_df.sort_values(['population'],ascending=False).head(10)

,city_name,population
1341,Shanghai,22315474
771,Istanbul,13710512
1527,Mumbai,12442373
479,Moskva,11979529
1340,Beijing,11716620
2810,São Paulo,11152344
1342,Tianjin,11090314
1064,Guangzhou,11071424
1582,Delhi,11034555
1067,Shenzhen,10358381


3) 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [12]:
ethnicityList = []
pop_List = []
for country in document.findall('country'):
    for population in reversed(country.findall('population')):
        pop_List.append(int(population.text))
        for ethnicity in country.findall('ethnicgroup'):
            ethnicityList.append((int(population.text), float(ethnicity.attrib['percentage']), ethnicity.text))
        break

df= pd.DataFrame(ethnicityList, columns=['population', 'percentage', 'ethnicity'])
df['Ethnic Population'] = (df.population * df.percentage)/100
del df['population']
del df['percentage']
df.groupby('ethnicity').sum().sort_values(by='Ethnic Population', ascending=False).head(10)

,Ethnic Population
ethnicity,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


In [13]:
import numpy as np

4) name and country of a) longest river

In [14]:
rivers = pd.DataFrame([[r.find('name').text, np.nan 
    if r.find('length') is None else float(r.find('length').text),r.get('country')] 
        for r in document.findall('river')],
            columns=['River', 'Length','Country code'])
rivers.sort_values('Length',ascending = False)[0:1]

,River,Length,Country code
174,Amazonas,6448.0,CO BR PE


4)  b) largest lake 

In [15]:
lakes = pd.DataFrame([[lakes_.find('name').text, np.nan 
    if lakes_.find('area') is None else float(lakes_.find('area').text),lakes_.get('country')] 
        for lakes_ in document.findall('lake')],
            columns=['Lakes', 'Area','Country code'])
lakes.sort_values('Area',ascending = False)[0:1]

,Lakes,Area,Country code
54,Caspian Sea,386400.0,R AZ KAZ IR TM


4) c) airport at highest elevation

In [16]:
airport = pd.DataFrame([[airports.find('name').text, np.nan 
    if airports.find('elevation') is None else float(airports.find('elevation').text),airports.get('country')] 
        for airports in document.findall('lake')],
            columns=['Airport', 'Elevation','Country code'])
airport.sort_values('Elevation',ascending = False)[0:1]

,Airport,Elevation,Country code
134,Licancabur Crater Lake,5600.0,RCH BOL
